## -- Step 1 -  Import necessary libraries --- 


In [ ]:
from openai import OpenAI
import hopsworks
from elasticsearch import Elasticsearch

## -- Step 2 - Define Hopsworks and Google API keys and connect to Hopsworks ---

In [ ]:
# Hopsworks and Google API keys
HOPSWORKS_API_KEY = exclude.key.HOPSWORKS_API_KEY
FEATURE_GROUP_NAME = "bars_near_london_bridge"
FEATURE_GROUP_EMBEDDING_NAME = "venue_description_embeddings"
FEATURE_GROUP_EMBEDDING_VERSION = 2
FEATURE_GROUP_VERSION = 3
GOOGLE_API_KEY = exclude.key.GOOGLE_API_KEY

ELASTIC_CLOUD_ID = exclude.key.ELASTIC_CLOUD_ID
ELASTIC_API_KEY = exclude.key.ELASTIC_API_KEY

# Connect to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

## -- Step 2 - Setup SentenceTransformer

In [22]:
from sentence_transformers import SentenceTransformer

# Define the SentenceTransformer model name
MODEL_NAME = 'all-MiniLM-L6-v2'
model = SentenceTransformer(
    MODEL_NAME,
)

## -- Step 2 - Define Embedding Query --- 

In [ ]:
sentences = ["Beer garden is open.", "There are lots of craft beer Available.", "Late Opening hours"]
embedding_query = model.encode(sentences)

### Step 3: Setup Elastic


In [ ]:
es = Elasticsearch(cloud_id=ELASTIC_CLOUD_ID, api_key=ELASTIC_API_KEY) 

es.info()



### Step 4: Search Elastic


results = es.search(
    knn={
        'field': 'embeddings',
        'query_vector': embedding_query,
        'num_candidates': 2,
        'k': 2,
    },
    size=2
)
